In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
import pandas as pd
import numpy as np
!pip install 'wget'
import wget

  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [5]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']
for url in urls:
    filename = wget.download(url)

In [6]:
conf_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_US.csv')
deaths_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,1,1,2,3,3,4,4,4,4,4
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,0,0,0,1,1,1,1,1,1,1
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,3,6,8,11,12,15,18,20,21,23
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,0,0,0,0,0,0,0,1,1,1


In [7]:
dates = conf_df.columns[12:]

conf_df_long = conf_df.melt(id_vars=['FIPS','Province_State', 'Country_Region', 'Lat', 'Long_'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['FIPS','Province_State', 'Country_Region', 'Lat', 'Long_'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

full_table = pd.concat([conf_df_long, deaths_df_long['Deaths']], 
                       axis=1, sort=False)

full_table.head()

,FIPS,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths
0,60.0,American Samoa,US,-14.2710,-170.1320,1/23/20,0,0
1,66.0,Guam,US,13.4443,144.7937,1/23/20,0,0
2,69.0,Northern Mariana Islands,US,15.0979,145.6739,1/23/20,0,0
3,72.0,Puerto Rico,US,18.2208,-66.5901,1/23/20,0,0
4,78.0,Virgin Islands,US,18.3358,-64.8963,1/23/20,0,0


In [9]:
#limit to United States
us_only = full_table[(full_table['Country_Region']=='US')]
us_only.head()

,FIPS,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths
0,60.0,American Samoa,US,-14.2710,-170.1320,1/23/20,0,0
1,66.0,Guam,US,13.4443,144.7937,1/23/20,0,0
2,69.0,Northern Mariana Islands,US,15.0979,145.6739,1/23/20,0,0
3,72.0,Puerto Rico,US,18.2208,-66.5901,1/23/20,0,0
4,78.0,Virgin Islands,US,18.3358,-64.8963,1/23/20,0,0


In [29]:
#get state level data first
state_data = us_only[us_only['FIPS'].apply(lambda x: len(str(x)) == 4)]
state_data.head()

,FIPS,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths
0,60.0,American Samoa,US,-14.2710,-170.1320,1/23/20,0,0
1,66.0,Guam,US,13.4443,144.7937,1/23/20,0,0
2,69.0,Northern Mariana Islands,US,15.0979,145.6739,1/23/20,0,0
3,72.0,Puerto Rico,US,18.2208,-66.5901,1/23/20,0,0
4,78.0,Virgin Islands,US,18.3358,-64.8963,1/23/20,0,0


In [43]:
#get county level data 
county_data = us_only[us_only['FIPS'].apply(lambda x: len(str(x)) > 4)]
county_data.head()

,FIPS,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths
5,1001.0,Alabama,US,32.539527,-86.644082,1/23/20,0,0
6,1003.0,Alabama,US,30.727750,-87.722071,1/23/20,0,0
7,1005.0,Alabama,US,31.868263,-85.387129,1/23/20,0,0
8,1007.0,Alabama,US,32.996421,-87.125115,1/23/20,0,0
9,1009.0,Alabama,US,33.982109,-86.567906,1/23/20,0,0


In [33]:
#bring in csv to get county names
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv']
for url in urls:
    filename = wget.download(url)
filename

'UID_ISO_FIPS_LookUp_Table.csv'

In [44]:
lookup = pd.read_csv('UID_ISO_FIPS_LookUp_Table.csv')
lookup.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [45]:
#merge to get the county name on the county level records
county_data = pd.merge(county_data, lookup, on='FIPS')
county_data.head()

,FIPS,Province_State_x,Country_Region_x,Lat_x,Long__x,Date,Confirmed,Deaths,UID,iso2,iso3,code3,Admin2,Province_State_y,Country_Region_y,Lat_y,Long__y,Combined_Key,Population
0,1001.0,Alabama,US,32.539527,-86.644082,1/23/20,0,0,84001001,US,USA,840.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0
1,1001.0,Alabama,US,32.539527,-86.644082,1/24/20,0,0,84001001,US,USA,840.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0
2,1001.0,Alabama,US,32.539527,-86.644082,1/25/20,0,0,84001001,US,USA,840.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0
3,1001.0,Alabama,US,32.539527,-86.644082,1/26/20,0,0,84001001,US,USA,840.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0
4,1001.0,Alabama,US,32.539527,-86.644082,1/27/20,0,0,84001001,US,USA,840.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0


In [46]:
#select just a few of the necessary colums & cleanup the column names
county_data = county_data[['FIPS','Admin2','Province_State_x','Country_Region_x', 'Date', 'Lat_x', 'Long__x', 'Confirmed', 'Deaths']]
county_data['County']=county_data['Admin2']
county_data['Province_State'] = county_data['Province_State_x']
county_data['Country_Region'] = county_data['Country_Region_x']
county_data['Lat'] = county_data['Lat_x']
county_data['Long'] = county_data['Long__x']
county_data = county_data[['FIPS','County','Province_State','Country_Region', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths']]
county_data.head()

,FIPS,County,Province_State,Country_Region,Date,Lat,Long,Confirmed,Deaths
0,1001.0,Autauga,Alabama,US,1/23/20,32.539527,-86.644082,0,0
1,1001.0,Autauga,Alabama,US,1/24/20,32.539527,-86.644082,0,0
2,1001.0,Autauga,Alabama,US,1/25/20,32.539527,-86.644082,0,0
3,1001.0,Autauga,Alabama,US,1/26/20,32.539527,-86.644082,0,0
4,1001.0,Autauga,Alabama,US,1/27/20,32.539527,-86.644082,0,0


In [47]:
state_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 243979
Data columns (total 8 columns):
FIPS              380 non-null float64
Province_State    380 non-null object
Country_Region    380 non-null object
Lat               380 non-null float64
Long_             380 non-null float64
Date              380 non-null object
Confirmed         380 non-null int64
Deaths            380 non-null int64
dtypes: float64(3), int64(2), object(3)
memory usage: 26.7+ KB


In [48]:
county_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246696 entries, 0 to 246695
Data columns (total 9 columns):
FIPS              246696 non-null float64
County            246544 non-null object
Province_State    246696 non-null object
Country_Region    246696 non-null object
Date              246696 non-null object
Lat               246696 non-null float64
Long              246696 non-null float64
Confirmed         246696 non-null int64
Deaths            246696 non-null int64
dtypes: float64(3), int64(2), object(4)
memory usage: 18.8+ MB


In [49]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'county_data_jh.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': '4b11ac40-adf0-437a-b9a6-a0cc759fa95e'}

In [50]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'state_data_jh.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': '76747afb-787c-4493-9315-90f89a6022fb'}